The Jacobian Transpose method is an inverse kinematic method that tries to find joint values to obtain an end effector position. The algorithm tries to approximate the inverse jacobian with the jacobian transpose to try and obtain a solution for the joint values.

In [364]:
import numpy as np
import sympy as sym
import math

In [365]:
# SOME HARD CODED VALUES FOR ROBOT PARAMETERS
z = 0.3
l1 = 0.5
l2 = 0.5
r1 = 0.2
r2 = 0.25
r3 = 0.3
phi_value = 0
theta1_value = 1.5714
theta2_value = 0
theta3_value = 1.5714
starting_angles = sym.Matrix([phi_value, theta1_value, theta2_value, theta3_value])
phi, theta1, theta2, theta3 = sym.symbols("phi theta1 theta2 theta3")


In [366]:
# MAKE A DH PARAMETER TABLE
# d, alpha, r, theta
dh_parameters = sym.Matrix(
  [
   [0, 0, 0, 0],
   [z + r1, 1.57, 0, phi],
   [0, 0, r1 + l1 + r2, theta1],
   [0, 0, r2 + l2 + r3, theta2],
   [0, 0, r3, theta3]
  ] 
)
print(dh_parameters)


Matrix([[0, 0, 0, 0], [0.500000000000000, 1.57000000000000, 0, phi], [0, 0, 0.950000000000000, theta1], [0, 0, 1.05000000000000, theta2], [0, 0, 0.300000000000000, theta3]])


In [367]:

# MAKE A DH MATRIX FOR THE ROBOT

# USING THE PARAMETERS 
dh_matrix = np.diag(np.ones((4)))

print(dh_parameters.shape[0])
for i in range(dh_parameters.shape[0]):
  
  trans_z = sym.Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, dh_parameters[i, 0]],
    [0, 0, 0, 1]
  ])
  dh_matrix = dh_matrix * trans_z
  rot_z = sym.Matrix([
    [sym.cos(dh_parameters[i, 3]), -1 * sym.sin(dh_parameters[i, 3]), 0, 0],
    [sym.sin(dh_parameters[i, 3]), sym.cos(dh_parameters[i, 3]), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0 ,1]
  ])
  dh_matrix = dh_matrix * rot_z
  trans_x = sym.Matrix([
    [1, 0, 0, dh_parameters[i, 2]],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
  ])
  dh_matrix = dh_matrix * trans_x 
  rot_x = sym.Matrix([
    [1, 0, 0, 0],
    [0, sym.cos(dh_parameters[i, 1]), -1 * sym.sin(dh_parameters[i, 1]), 0],
    [0, sym.sin(dh_parameters[i, 1]), sym.cos(dh_parameters[i, 1]), 0],
    [0, 0, 0, 1]
  ])
  dh_matrix =  dh_matrix * rot_x
  
  
print(dh_matrix) 

5
Matrix([[(-(-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*sin(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*cos(theta2))*sin(theta3) + ((-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*cos(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*sin(theta2))*cos(theta3), (-(-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*sin(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*cos(theta2))*cos(theta3) - ((-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*cos(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*sin(theta2))*sin(theta3), 0.999999682931835*sin(phi), 0.3*(-(-0.000796326710733263*sin(phi)*sin(theta1) + 1.0*cos(phi)*cos(theta1))*sin(theta2) + (-0.000796326710733263*sin(phi)*cos(theta1) - 1.0*sin(theta1)*cos(phi))*cos(theta2))*sin(theta3) + 0.3*((-0.00079632671073326

In [368]:
# GET START POSITION
position_function = dh_matrix[0:3, -1]
start_position = position_function.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
current_position = start_position
print(start_position)

Matrix([[-0.301207127684100], [0.00159236469867950], [2.49963679772224]])


In [369]:
# SET END POSITION

target_position = sym.Matrix([1, 1, 1])


In [370]:
# ALGORITHM

# CALCULATE THE JACOBIAN
angles = sym.Matrix([phi, theta1, theta2, theta3])
# 
jacobian_matrix = position_function.jacobian(angles)

# CALCULATE THE JACOBIAN TRANSPOSE
jacobian_transpose = jacobian_matrix.transpose()

difference_position = target_position - position_function

jte = jacobian_transpose * difference_position
jjte = jacobian_matrix * jte

alpha = jte.dot(jte) / jjte.dot(jjte)



In [371]:
epsilon_error = 0.1
# USE MSE AS A METRIC FOR CONVERGENCE
def mse(target_position, calculated_position):
  sum = 0.0
  for i in range(len(target_position)):
    sum += (target_position[i] - calculated_position[i]) ** 2
  return(math.sqrt(sum))  

In [372]:

while(mse(target_position, current_position) > epsilon_error):
  alpha_value = alpha.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
  delta_theta = alpha_value * jacobian_transpose * difference_position

  jacobian_matrix_value = jacobian_matrix.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
  delta_theta_value = delta_theta.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
    
  phi_value += delta_theta_value[0]
  theta1_value += delta_theta_value[1]
  theta2_value += delta_theta_value[2]
  theta3_value += delta_theta_value[3]
  
  current_position = position_function.evalf(subs={phi: phi_value, theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
  print(current_position)
